In [1]:
import requests
import json 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import functools

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests

#### Federal Reserve

In [2]:
federal_reserve=pd.read_csv("Federal_Reserve.csv")
federal_reserve.name=federal_reserve.name.astype("datetime64")
federal_reserve

,name,volume
0,2022-02-14,21098
1,2022-02-15,39253
2,2022-02-16,52920
3,2022-02-17,33123
4,2022-02-18,24411
5,2022-02-19,10074
6,2022-02-20,7916
7,2022-02-21,11244
8,2022-02-22,16364
9,2022-02-23,15286


In [3]:
BASE_URL = "https://www.alphavantage.co/query"
from_currency="TRY"
to_currency="JPY"

data_ = {"apikey": "XNH4BI4LYB40JJAI",
         "function":"FX_DAILY",
         "from_symbol":from_currency,
         "to_symbol": to_currency,
         "datatype":"json",
        "outputsize":"compact"}

response = requests.get(BASE_URL, params = data_).json()

data = pd.DataFrame(response['Time Series FX (Daily)'])
data = data.T[["4. close"]].copy()
data.reset_index(inplace=True)
data.rename(columns = {"4. close": "price","index":"date"}, inplace = True)
data.date=data.date.astype("datetime64")
data.price=data.price.astype("float64")
data

,date,price
0,2022-03-18,8.008
1,2022-03-17,8.015
2,2022-03-16,8.070
3,2022-03-15,8.000
4,2022-03-14,7.945
...,...,...
95,2021-11-05,11.652
96,2021-11-04,11.699
97,2021-11-03,11.800
98,2021-11-02,11.853


In [4]:
correl_test= pd.merge(federal_reserve, data, how ='inner', left_on =['name'],right_on=['date'])
correl_test.corr()

,volume,price
volume,1.000000,0.250552
price,0.250552,1.000000


In [5]:
## Offset

In [6]:
offset = 7
df_offset = correl_test.copy()

df_offset[["price"]] = df_offset[[ "price"]].shift(-offset)                      
df_offset.dropna(inplace = True)

df_offset.corr(method = "spearman")

,volume,price
volume,1.000000,0.453571
price,0.453571,1.000000


In [7]:
offset = 1
df_offset = correl_test.copy()

df_offset[["price"]] = df_offset[[ "price"]].shift(-offset)                      
df_offset.dropna(inplace = True)

df_offset.corr(method = "spearman")

,volume,price
volume,1.000000,0.275325
price,0.275325,1.000000


In [8]:
result = adfuller(correl_test["price"].values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -0.139312
p-value: 0.945379
Critical values:
	1%: -4.138
	5%: -3.155
	10%: -2.714


In [9]:
result = adfuller(correl_test["price"].pct_change(periods=1).dropna().values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -7.786820
p-value: 0.000000
Critical values:
	1%: -4.138
	5%: -3.155
	10%: -2.714


In [10]:
df_diff = correl_test[["price","volume"]].pct_change(periods=1).dropna()
df_diff.corr(method = "spearman")

,price,volume
price,1.000000,-0.022078
volume,-0.022078,1.000000


In [11]:
maxlag=1
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var for var in variables]
    df.index = [var for var in variables]
    return df

In [12]:
grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])

,volume,price
volume,1.0000,0.2168
price,0.2667,1.0000


In [13]:
#Feds

In [14]:
feds=pd.read_csv("feds.csv")
feds.date=feds.date.astype("datetime64")
feds

,date,volume
0,2022-02-14,477
1,2022-02-15,290
2,2022-02-16,275
3,2022-02-17,380
4,2022-02-18,425
5,2022-02-19,257
6,2022-02-20,199
7,2022-02-21,196
8,2022-02-22,233
9,2022-02-23,204


In [15]:
BASE_URL = "https://www.alphavantage.co/query"
from_currency="TRY"
to_currency="JPY"

data_ = {"apikey": "XNH4BI4LYB40JJAI",
         "function":"FX_DAILY",
         "from_symbol":from_currency,
         "to_symbol": to_currency,
         "datatype":"json",
        "outputsize":"compact"}

response = requests.get(BASE_URL, params = data_).json()

data = pd.DataFrame(response['Time Series FX (Daily)'])
data = data.T[["4. close"]].copy()
data.reset_index(inplace=True)
data.rename(columns = {"4. close": "price","index":"date"}, inplace = True)
data.date=data.date.astype("datetime64")
data.price=data.price.astype("float64")
data

,date,price
0,2022-03-18,8.008
1,2022-03-17,8.015
2,2022-03-16,8.070
3,2022-03-15,8.000
4,2022-03-14,7.945
...,...,...
95,2021-11-05,11.652
96,2021-11-04,11.699
97,2021-11-03,11.800
98,2021-11-02,11.853


In [16]:
correl_test= pd.merge(feds, data, how ='inner', on =['date'])
correl_test.corr()

,volume,price
volume,1.000000,0.479213
price,0.479213,1.000000


In [17]:
offset = 7
df_offset = correl_test.copy()

df_offset[["price"]] = df_offset[[ "price"]].shift(-offset)                      
df_offset.dropna(inplace = True)

df_offset.corr(method = "spearman")

,volume,price
volume,1.000000,0.564286
price,0.564286,1.000000


In [18]:
offset = 1
df_offset = correl_test.copy()

df_offset[["price"]] = df_offset[[ "price"]].shift(-offset)                      
df_offset.dropna(inplace = True)

df_offset.corr(method = "spearman")

,volume,price
volume,1.00000,0.51039
price,0.51039,1.00000


In [19]:
result = adfuller(correl_test["price"].values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -0.139312
p-value: 0.945379
Critical values:
	1%: -4.138
	5%: -3.155
	10%: -2.714


In [20]:
result = adfuller(correl_test["price"].pct_change(periods=1).dropna().values)
print('ADF Statistics: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistics: -7.786820
p-value: 0.000000
Critical values:
	1%: -4.138
	5%: -3.155
	10%: -2.714


In [21]:
df_diff = correl_test[["price","volume"]].pct_change(periods=1).dropna()
df_diff.corr(method = "spearman")

,price,volume
price,1.000000,-0.145455
volume,-0.145455,1.000000


In [22]:
maxlag=1
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var for var in variables]
    df.index = [var for var in variables]
    return df

In [23]:
grangers_causation_matrix(df_diff, variables = [ 'volume', 'price'])

,volume,price
volume,1.0000,0.0092
price,0.2318,1.0000
